Limpiar Y Normalizar Datos

In [ ]:
import requests
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import zipfile
import os
import glob
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from io import StringIO
# Definir una función para cargar y procesar cada archivo individualmente
def cargar_y_procesar_archivo(file_path):
    # Cargar el archivo CSV
    df = pd.read_csv(file_path, dtype={'Hora': 'int32','Fecha': 'str','Pais': 'category','Unidad': 'category','Tipo_Oferta': 'category','Energía_Compra_Venta': 'str','Precio_Compra_Venta': 'str','Ofertada_Casada': 'float32'})
    # Inicializar un DataFrame vacio para almacenar los datos combinados
    data_frame_combinado = pd.DataFrame() 
    # Renombrar las columnas
    data_frame_combinado.columns = ['Hora', 'Fecha', 'Pais', 'Unidad', 'Tipo_Oferta', 'Energía_Compra_Venta','Precio_Compra_Venta', 'Ofertada_Casada', 'Unnamed']
    # Eliminar la columna 'Unnamed'
    data_frame_combinado.drop(columns=['Unnamed'], inplace=True)
    # Modificar tipos de datos:
    #1 'Hora' modificar la hora para números enteros.
    data_frame_combinado['Hora'] = data_frame_combinado['Hora'].astype(int)
    #2 'Fecha' modificar las fechas actuales de los archivos.csv 
    data_frame_combinado['Fecha'] = pd.to_datetime(data_frame_combinado['Fecha'], format='%d/%m/%Y')
    #3 'Energía_Compra_Venta' modificar los valores flotantes despues de reemplazar comas por puntos.
    data_frame_combinado['Energía_Compra_Venta'] = data_frame_combinado['Energía_Compra_Venta'].str.replace(',', '.').astype(float)
    #4 'Precio_Compra_Venta' modificar los valores flotantes despues de reemplazar comas por puntos.
    data_frame_combinado['Precio_Compra_Venta'] = data_frame_combinado['Precio_Compra_Venta'].str.replace('.', '').str.replace(',', '.').astype(float)
    # A continuación,se eliminaran los valores límites que nos estaban distorsionando la señal de precios del OMIE.
    # Creación de un data_frame_filtrado, Nuestra condición de operación de datos es entre un intervalo de precios de [>-4€/Mwh,<1500€/Mwh] 
    data_frame_filtrado = data_frame_combinado[(data_frame_combinado['Precio_Compra_Venta'] >= -4) & (data_frame_combinado['Precio_Compra_Venta'] <= 1500)]
    #verificación rápida de los valores filtrados procesado correctamente.
    print(data_frame_combinado.head())
return def





Codigo Optimizado con ChunkSize /Independiente

In [ ]:
# Código Chunksize se utiliza si los archivos CSV son muy grandes, podemos leerlos en trozos (chunks) y procesarlos por partes.
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import time
from dask import dataframe as dd
# Definir una función para cargar y procesar cada archivo individualmente
def cargar_y_procesar_archivo(file_path):
    """Carga y procesa un archivo CSV especificado."""
    # Leer archivo CSV en chunks
    chunk_list = []
    for chunk in pd.read_csv(file_path, dtype={'Hora': 'int32','Fecha': 'str','Pais': 'category','Unidad': 'category','Tipo_Oferta': 'category','Energía_Compra_Venta': 'str','Precio_Compra_Venta': 'str','Ofertada_Casada': 'float32'}, chunksize=100000):
        # Procesar cada chunk
        chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%d/%m/%Y')
        chunk['Energía_Compra_Venta'] = chunk['Energía_Compra_Venta'].str.replace(',', '.').astype('float32')
        chunk['Precio_Compra_Venta'] = chunk['Precio_Compra_Venta'].str.replace('.', '').str.replace(',', '.').astype('float32')
        # Filtrar valores límites de Precio_Compra_Venta
        chunk = chunk[(chunk['Precio_Compra_Venta'] >= -4) & (chunk['Precio_Compra_Venta'] <= 1500)]
        chunk_list.append(chunk)
    
    return pd.concat(chunk_list, ignore_index=True)
